<a href="https://colab.research.google.com/github/YolandaMDavis/cs109a-fall2020/blob/socio-economic/Boston_Covid_Analysis_plus_Socio_Eco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# How to use Geohash - https://bigfastblog.com/geohash-intro

!pip install pygeohash

  Created wheel for pygeohash: filename=pygeohash-1.2.0-py2.py3-none-any.whl size=6162 sha256=5d477c779a87d60221c494cea75a71d764e382e5b02fe760b0bb6c8bd7403c15
  Stored in directory: /root/.cache/pip/wheels/3f/5f/14/989d83a271207dda28232746d63e737a2dbd88ea7f7a9db807
Successfully built pygeohash


In [3]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pygeohash as pgh
DATA_ROOT = '/content/drive/My Drive/Final-Project-for-CS109a/covid_analysis/'

In [ ]:
boston_covid_neighborhood_df = pd.read_csv(DATA_ROOT + 'boston_neighborhood_covid.csv')
boston_covid_neighborhood_df["total positive"] = boston_covid_neighborhood_df["total tested"] * boston_covid_neighborhood_df["cumulative positive"]
boston_covid_neighborhood_df = boston_covid_neighborhood_df.rename(columns={"neigborhood":"neighborhood"})
boston_covid_neighborhood_df.head()


,neighborhood,total tested,cumulative positive,total positive
0,East Boston,19590,0.13600,2664.2400
1,Mattapan,8652,0.11200,969.0240
2,Hyde Park,12726,0.11100,1412.5860
3,Dorchester,48850,0.10751,5251.8635
4,Roslindale,12005,0.08200,984.4100


In [ ]:
boston_districts_df = pd.read_excel(DATA_ROOT + 'Boston-Neighborhoods-to-Districts.xlsx')
boston_districts_df

,Unnamed: 0,neighborhood,district
0,0,Allston,D14
1,1,Back Bay,D4
2,2,Beacon Hill,A1
3,3,Brighton,D14
4,4,Charlestown,A15
5,5,Dorchester,C11
6,6,Downtown,A1
7,7,East Boston,A7
8,8,Fenway,D4
9,9,Harbor Islands,NaN


In [ ]:
boston_covid_districts = pd.merge(boston_covid_neighborhood_df,boston_districts_df, on="neighborhood").drop(columns=["Unnamed: 0"])
boston_covid_districts

,neighborhood,total tested,cumulative positive,total positive,district
0,East Boston,19590,0.136,2664.240,A7
1,Roslindale,12005,0.082,984.410,E5
2,Roxbury,23736,0.062,1471.632,B2
3,West Roxbury,9356,0.061,570.716,E5
4,South End,20060,0.057,1143.420,D4
5,Jamaica Plain,19720,0.049,966.280,E13
6,South Boston,16450,0.047,773.150,C6
7,Allston,12753,0.042,535.626,D14
8,Brighton,18697,0.042,785.274,D14
9,Charlestown,7060,0.039,275.340,A15


In [ ]:
boston_covid_districts = boston_covid_districts.groupby("district").sum()[["total positive","total tested","cumulative positive"]]
boston_covid_districts

,total positive,total tested,cumulative positive
district,,,
A1,476.523,17649,0.108
A15,275.340,7060,0.039
A7,2664.240,19590,0.136
B2,1471.632,23736,0.062
C6,773.150,16450,0.047
D14,1320.900,31450,0.084
D4,1971.395,70570,0.098
E13,966.280,19720,0.049
E5,1555.126,21361,0.143


In [ ]:
boston_crime_df = pd.read_csv(DATA_ROOT + 'boston_crime_2020_latest.csv')
boston_crime_df.head()


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2,6,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,TESTTEST2,423,NaN,ASSAULT - AGGRAVATED,External,,0,2019-10-16 00:00:00,2019,10,Wednesday,0,NaN,RIVERVIEW DR,NaN,NaN,"(0.00000000, 0.00000000)"
1,S97333701,3301,NaN,VERBAL DISPUTE,C6,915,0,2020-07-18 14:34:00,2020,7,Saturday,14,NaN,MARY BOYLE WAY,42.330813,-71.051368,"(42.33081300, -71.05136800)"
2,S47513131,2647,NaN,THREATS TO DO BODILY HARM,E18,530,0,2020-06-24 10:15:00,2020,6,Wednesday,10,NaN,READVILLE ST,42.239491,-71.135954,"(42.23949100, -71.13595400)"
3,I92102201,3301,NaN,VERBAL DISPUTE,E13,583,0,2019-12-20 03:08:00,2019,12,Friday,3,NaN,DAY ST,42.325122,-71.107779,"(42.32512200, -71.10777900)"
4,I92097173,3115,NaN,INVESTIGATE PERSON,C11,355,0,2019-10-23 00:00:00,2019,10,Wednesday,0,NaN,GIBSON ST,42.297555,-71.059709,"(42.29755500, -71.05970900)"


In [ ]:
crime_violence_ind = pd.read_csv(DATA_ROOT + 'boston_crime_violence_ind.csv')
crime_violence_ind["OFFENSE_TYPE"].unique()

array(['Assault and/or Battery', 'Other', 'Animal Crime',
       'MV Accident/Property Damage', 'Theft/Robbery/B&E',
       'Child Abuse/Endangerment', 'Harassment',
       'Murder/Manslaughter/Homocide', 'Drugs/Alcohol', 'Weapon',
       'Sex Offense', 'Forgery/Fraud', 'Kidnapping', 'Missing Person',
       'Prisoner Offense', 'Prostitution', 'Val', 'Violation of Order',
       'Warrant'], dtype=object)

In [ ]:
# Select 2020 crimes
boston_crime_2020_df = boston_crime_df[boston_crime_df["YEAR"] == 2020].drop(columns=["OFFENSE_CODE_GROUP"])

# Merge with Covid Data
boston_crime_2020_df = pd.merge(boston_crime_2020_df,boston_covid_districts, left_on="DISTRICT", right_on="district")

# Merge with new Violent Crime Indicator
boston_crime_2020_df = pd.merge(boston_crime_2020_df,crime_violence_ind, left_on="OFFENSE_DESCRIPTION", right_on="OFFENSE_DESCRIPTION")

# Set any shooting indicator as violent crime
filter = boston_crime_2020_df["SHOOTING"] == 1
boston_crime_2020_df.loc[filter,"VIOLENT_CRIME"] = 1


In [ ]:
boston_crime_2020_df = boston_crime_2020_df.rename(columns={"total positive": "COVID_POSITIVE_COUNT", "total tested": "COVID_TESTED_COUNT", "cumulative positive": "COVID_POSITIVE_RATE","Lat":"LATITUDE","Long":"LONGITUDE","Location":"LOCATION"})
boston_crime_2020_df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,LATITUDE,LONGITUDE,LOCATION,COVID_POSITIVE_COUNT,COVID_TESTED_COUNT,COVID_POSITIVE_RATE,VIOLENT_CRIME,OFFENSE_TYPE
0,S97333701,3301,VERBAL DISPUTE,C6,915,0,2020-07-18 14:34:00,2020,7,Saturday,14,NaN,MARY BOYLE WAY,42.330813,-71.051368,"(42.33081300, -71.05136800)",773.15,16450,0.047,0,Other
1,202080441,3301,VERBAL DISPUTE,C6,231,0,2020-10-31 02:41:00,2020,10,Saturday,2,NaN,E SEVENTH ST,NaN,NaN,"(0.00000000, 0.00000000)",773.15,16450,0.047,0,Other
2,202080246,3301,VERBAL DISPUTE,C6,914,0,2020-10-30 10:51:00,2020,10,Friday,10,NaN,GENERAL LAWRENCE J LOGAN WAY,NaN,NaN,"(0.00000000, 0.00000000)",773.15,16450,0.047,0,Other
3,202080065,3301,VERBAL DISPUTE,C6,186,0,2020-10-29 16:30:00,2020,10,Thursday,16,NaN,BOSTON ST,42.327827,-71.058000,"(42.32782700, -71.05800000)",773.15,16450,0.047,0,Other
4,202079731,3301,VERBAL DISPUTE,C6,936,0,2020-10-28 13:52:00,2020,10,Wednesday,13,NaN,ORTON-MAROTTA WAY,42.337318,-71.053832,"(42.33731800, -71.05383200)",773.15,16450,0.047,0,Other


In [ ]:
boston_crime_2020_df["GEOHASH"] = [pgh.encode(row["LATITUDE"],row["LONGITUDE"]) for idx, row in boston_crime_2020_df.iterrows()]
boston_crime_2020_df.sort_values(by=["GEOHASH"])

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,LATITUDE,LONGITUDE,LOCATION,COVID_POSITIVE_COUNT,COVID_TESTED_COUNT,COVID_POSITIVE_RATE,VIOLENT_CRIME,OFFENSE_TYPE,GEOHASH
8997,202004695,3115,INVESTIGATE PERSON,D4,623,0,2020-01-18 00:00:00,2020,1,Saturday,0,NaN,COMMONWEALTH AVENUE,NaN,NaN,"(0.00000000, 0.00000000)",1971.395,70570,0.098,0,Other,000000000000
28931,202080295,1102,FRAUD - FALSE PRETENSE / SCHEME,E13,574,0,2020-10-30 00:00:00,2020,10,Friday,0,NaN,WASHINGTON ST,NaN,NaN,"(0.00000000, 0.00000000)",966.280,19720,0.049,0,Forgery/Fraud,000000000000
14839,202048507,3831,M/V - LEAVING SCENE - PROPERTY DAMAGE,A1,,0,2020-07-09 12:00:00,2020,7,Thursday,12,NaN,SUDBURY ST,NaN,NaN,"(0.00000000, 0.00000000)",476.523,17649,0.108,0,MV Accident/Property Damage,000000000000
28932,202080216,1102,FRAUD - FALSE PRETENSE / SCHEME,E13,574,0,2020-10-30 09:02:00,2020,10,Friday,9,NaN,WASHINGTON ST,NaN,NaN,"(0.00000000, 0.00000000)",966.280,19720,0.049,0,Forgery/Fraud,000000000000
14836,202049782,3831,M/V - LEAVING SCENE - PROPERTY DAMAGE,A1,,0,2020-07-14 19:35:00,2020,7,Tuesday,19,NaN,SUDBURY ST,NaN,NaN,"(0.00000000, 0.00000000)",476.523,17649,0.108,0,MV Accident/Property Damage,000000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3208,202027061,3301,VERBAL DISPUTE,A7,2,0,2020-04-14 18:09:00,2020,4,Tuesday,18,NaN,WALDEMAR AVE,42.391462,-71.001341,"(42.39146200, -71.00134100)",2664.240,19590,0.136,0,Other,drt90m4khv6b
3178,202032567,3301,VERBAL DISPUTE,A7,2,0,2020-05-10 19:31:00,2020,5,Sunday,19,NaN,WALDEMAR AVE,42.391462,-71.001341,"(42.39146200, -71.00134100)",2664.240,19590,0.136,0,Other,drt90m4khv6b
32045,202028784,614,LARCENY THEFT FROM MV - NON-ACCESSORY,A7,2,0,2020-04-23 08:36:00,2020,4,Thursday,8,NaN,WALDEMAR AVE,42.391462,-71.001341,"(42.39146200, -71.00134100)",2664.240,19590,0.136,0,Theft/Robbery/B&E,drt90m4khv6b
30456,202074774,3006,SICK/INJURED/MEDICAL - PERSON,A7,2,0,2020-10-10 20:47:00,2020,10,Saturday,20,NaN,WALDEMAR AVE,42.391462,-71.001341,"(42.39146200, -71.00134100)",2664.240,19590,0.136,0,Other,drt90m4khv6b


In [4]:
#changing data path to main folder
data_path = "/content/drive/My Drive/Final-Project-for-CS109a/"

In [5]:
data = pd.read_excel(data_path + 'boston_neighborhood_demographics_2013-2017.xlsx', sheet_name="HH Income", skiprows=[0,1], header=0, skipfooter=3)

In [6]:
data.head()

,Unnamed: 0,Median Income,Total Households,"$14,999 and under",%,"$15,000 to $24,999",%.1,"$25,000 to $34,999",%.2,"$35,000 to $49,999",%.3,"$50,000 to $74,999",%.4,"$75,000 to $99,999",%.5,"$100,000 to $149,999",%.6,"$150,000 +",%.7
0,United States,57652.00,118825921,13710365,0.115382,11637905,0.097941,11330288,0.095352,15412493,0.129706,21000314,0.176732,14636046,0.123172,16701857,0.140557,14396653,0.121158
1,Massachusetts,74167.00,2585715,269677,0.104295,203309,0.078628,186707,0.072207,255677,0.098881,390219,0.150913,317494,0.122788,451683,0.174684,510949,0.197605
2,Boston,62021.00,263229,47236,0.179448,22654,0.086062,18413,0.069950,24161,0.091787,37621,0.142921,26157,0.099370,39811,0.151241,47176,0.179220
3,Allston,46982.76,6457,1453,0.225027,584,0.090444,603,0.093387,781,0.120954,1042,0.161375,544,0.084250,912,0.141242,538,0.083320
4,Back Bay,102070.55,9824,1402,0.142712,381,0.038783,350,0.035627,582,0.059243,1237,0.125916,879,0.089475,1586,0.161441,3407,0.346804


In [7]:
median_income = data.iloc[3:, :2]

In [8]:
median_income = median_income.reset_index(drop=True)
median_income.columns = ['NEIGHBORHOOD', 'MEDIAN_INCOME']
median_income

,NEIGHBORHOOD,MEDIAN_INCOME
0,Allston,46982.76
1,Back Bay,102070.55
2,Beacon Hill,98069.24
3,Brighton,62041.20
4,Charlestown,103243.16
5,Dorchester,49662.36
6,Downtown,67367.09
7,East Boston,52935.36
8,Fenway,39549.84
9,Harbor Islands,NaN


In [9]:
crime_to_districts = pd.read_csv(data_path + 'Boston-Neighborhoods-to-Districts.csv')

In [10]:
income_by_district = pd.merge(median_income,crime_to_districts, left_on="NEIGHBORHOOD", right_on="neighborhood", copy=False).drop(columns=['neighborhood'])
income_by_district.columns = ['NEIGHBORHOOD', 'MEDIAN_INCOME', 'DISTRICT']
income_by_district

,NEIGHBORHOOD,MEDIAN_INCOME,DISTRICT
0,Allston,46982.76,D14
1,Back Bay,102070.55,D4
2,Beacon Hill,98069.24,A1
3,Brighton,62041.20,D14
4,Charlestown,103243.16,A15
5,Dorchester,49662.36,C11
6,Downtown,67367.09,A1
7,East Boston,52935.36,A7
8,Fenway,39549.84,D4
9,Harbor Islands,NaN,NaN


In [11]:
boston_income_districts = income_by_district.groupby("DISTRICT").mean()
boston_income_districts

,MEDIAN_INCOME
DISTRICT,
A1,89833.4675
A15,103243.1600
A7,52935.3600
B2,31714.3350
B3,48196.9000
C11,49662.3600
C6,121877.5550
D14,54511.9800
D4,65903.7975


**Single Family Households**, and **Youth Unemployment** numbers

In [12]:
demo_race = pd.read_excel(data_path + 'boston_neighborhood_demographics_2013-2017.xlsx', sheet_name="Race and Ethnicity", skiprows=[0,1,2], header=0, skipfooter=3)

In [13]:
race_perc_by_neighborhood = demo_race.iloc[3:, [0,3,5,7,9,11]]
race_perc_by_neighborhood.columns = ["NEIGHBORHOOD", "WHITE", "BLACK", "HISPANIC", "ASIAN", "OTHER"]
race_perc_by_neighborhood.head()

,NEIGHBORHOOD,WHITE,BLACK,HISPANIC,ASIAN,OTHER
3,Allston,0.541961,0.061664,0.138098,0.216909,0.041368
4,Back Bay,0.755447,0.041648,0.081426,0.098702,0.022777
5,Beacon Hill,0.834479,0.012614,0.069326,0.061840,0.021741
6,Brighton,0.650266,0.044163,0.108313,0.159139,0.038119
7,Charlestown,0.731972,0.058357,0.118406,0.071636,0.019629


In [14]:
race_by_district = pd.merge(race_perc_by_neighborhood,crime_to_districts, left_on="NEIGHBORHOOD", right_on="neighborhood")
race_by_district.drop(columns="neighborhood", inplace=True)
race_by_district

,NEIGHBORHOOD,WHITE,BLACK,HISPANIC,ASIAN,OTHER,district
0,Allston,0.541961,0.061664,0.138098,0.216909,0.041368,D14
1,Back Bay,0.755447,0.041648,0.081426,0.098702,0.022777,D4
2,Beacon Hill,0.834479,0.012614,0.069326,0.061840,0.021741,A1
3,Brighton,0.650266,0.044163,0.108313,0.159139,0.038119,D14
4,Charlestown,0.731972,0.058357,0.118406,0.071636,0.019629,A15
5,Dorchester,0.215249,0.453834,0.180933,0.094452,0.055531,C11
6,Downtown,0.563904,0.034298,0.060577,0.316250,0.024970,A1
7,East Boston,0.325667,0.025571,0.574151,0.038067,0.036545,A7
8,Fenway,0.601203,0.044573,0.115897,0.203141,0.035186,D4
9,Harbor Islands,0.537267,0.295031,0.027950,0.000000,0.139752,NaN


In [15]:
youth_employment = pd.read_excel(data_path + 'boston_neighborhood_demographics_2013-2017.xlsx', sheet_name="Labor Force", skiprows=range(34), header=0, skipfooter=3)

In [16]:
youth_employment_rate = youth_employment.iloc[3:, [0,2,4]]
youth_employment_rate.columns = ['NEIGHBORHOOD', '16-19-EMPLOYMENT', '20-34-EMPLOYMENT']
youth_employment_rate

,NEIGHBORHOOD,16-19-EMPLOYMENT,20-34-EMPLOYMENT
3,Allston,0.381298,0.610287
4,Back Bay,0.379937,0.816484
5,Beacon Hill,0.137931,0.936528
6,Brighton,0.401752,0.822183
7,Charlestown,0.377828,0.921227
8,Dorchester,0.411586,0.840373
9,Downtown,0.478317,0.733696
10,East Boston,0.308054,0.855628
11,Fenway,0.430455,0.639059
12,Harbor Islands,0.500000,0.250000


In [17]:
race_employment_by_district = pd.merge(race_by_district,youth_employment_rate, on="NEIGHBORHOOD")
race_employment_by_district = race_employment_by_district.groupby('district').mean()
race_employment_by_district

,WHITE,BLACK,HISPANIC,ASIAN,OTHER,16-19-EMPLOYMENT,20-34-EMPLOYMENT
district,,,,,,,
A1,0.727176,0.030437,0.080069,0.140236,0.022081,0.198180,0.827472
A15,0.731972,0.058357,0.118406,0.071636,0.019629,0.377828,0.921227
A7,0.325667,0.025571,0.574151,0.038067,0.036545,0.308054,0.855628
B2,0.272019,0.336937,0.250297,0.110655,0.030092,0.437154,0.710761
B3,0.067967,0.730087,0.153209,0.017705,0.031033,0.464945,0.819336
C11,0.215249,0.453834,0.180933,0.094452,0.055531,0.411586,0.840373
C6,0.791930,0.045022,0.069712,0.074954,0.018382,0.389850,0.888016
D14,0.596113,0.052914,0.123206,0.188024,0.039743,0.391525,0.716235
D4,0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266


In [18]:
race_employment_income_by_district = pd.merge(race_employment_by_district, boston_income_districts, left_index=True, right_index=True)
race_employment_income_by_district

,WHITE,BLACK,HISPANIC,ASIAN,OTHER,16-19-EMPLOYMENT,20-34-EMPLOYMENT,MEDIAN_INCOME
district,,,,,,,,
A1,0.727176,0.030437,0.080069,0.140236,0.022081,0.198180,0.827472,89833.4675
A15,0.731972,0.058357,0.118406,0.071636,0.019629,0.377828,0.921227,103243.1600
A7,0.325667,0.025571,0.574151,0.038067,0.036545,0.308054,0.855628,52935.3600
B2,0.272019,0.336937,0.250297,0.110655,0.030092,0.437154,0.710761,31714.3350
B3,0.067967,0.730087,0.153209,0.017705,0.031033,0.464945,0.819336,48196.9000
C11,0.215249,0.453834,0.180933,0.094452,0.055531,0.411586,0.840373,49662.3600
C6,0.791930,0.045022,0.069712,0.074954,0.018382,0.389850,0.888016,121877.5550
D14,0.596113,0.052914,0.123206,0.188024,0.039743,0.391525,0.716235,54511.9800
D4,0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975


In [20]:
# get whole database and combine
boston_crime = pd.read_csv(data_path + '2020_10_31_boston_crime.csv')
boston_crime_socio_eco = pd.merge(boston_crime, race_employment_income_by_district, left_on="DISTRICT", right_index=True).drop(columns=['OFFENSE_CODE_GROUP', 'UCR_PART'])
boston_crime_socio_eco.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2,6,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,Lat,Long,Location,WHITE,BLACK,HISPANIC,ASIAN,OTHER,16-19-EMPLOYMENT,20-34-EMPLOYMENT,MEDIAN_INCOME
1,S97333701,3301,VERBAL DISPUTE,C6,915,0,2020-07-18 14:34:00,2020,7,Saturday,14,MARY BOYLE WAY,42.330813,-71.051368,"(42.33081300, -71.05136800)",0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555
10,I20210846,3201,PROPERTY - LOST/ MISSING,C6,206,0,2020-02-09 00:30:00,2020,2,Sunday,0,SEAPORT BLVD,42.352756,-71.047481,"(42.35275600, -71.04748100)",0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555
12,I20207883,3115,INVESTIGATE PERSON,C6,914,0,2020-04-18 17:11:00,2020,4,Saturday,17,OLD COLONY AVE,42.325573,-71.052481,"(42.32557300, -71.05248100)",0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555
58,I192078622,619,LARCENY ALL OTHERS,C6,201,NaN,2019-09-29 03:04:00,2019,9,Sunday,3,W BROADWAY,42.340070,-71.052794,"(42.34006986, -71.05279420)",0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555
70,I192078606,3201,PROPERTY - LOST,C6,200,NaN,2019-09-29 02:21:00,2019,9,Sunday,2,W BROADWAY,42.341288,-71.054679,"(42.34128751, -71.05467933)",0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555


In [ ]:
# merge this data with 2020 data above
# boston_crime_2020_df_new = pd.merge(boston_crime_2020_df,race_employment_income_by_district, left_on="DISTRICT", right_index=True)
# boston_crime_2020_df_new.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,LATITUDE,LONGITUDE,LOCATION,COVID_POSITIVE_COUNT,COVID_TESTED_COUNT,COVID_POSITIVE_RATE,VIOLENT_CRIME,OFFENSE_TYPE,GEOHASH,WHITE,BLACK,HISPANIC,ASIAN,OTHER,16-19-EMPLOYMENT,20-34-EMPLOYMENT,MEDIAN_INCOME
0,S97333701,3301,VERBAL DISPUTE,C6,915,0,2020-07-18 14:34:00,2020,7,Saturday,14,NaN,MARY BOYLE WAY,42.330813,-71.051368,"(42.33081300, -71.05136800)",773.15,16450,0.047,0,Other,drt2z4jfz5kp,0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555
1,202080441,3301,VERBAL DISPUTE,C6,231,0,2020-10-31 02:41:00,2020,10,Saturday,2,NaN,E SEVENTH ST,NaN,NaN,"(0.00000000, 0.00000000)",773.15,16450,0.047,0,Other,000000000000,0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555
2,202080246,3301,VERBAL DISPUTE,C6,914,0,2020-10-30 10:51:00,2020,10,Friday,10,NaN,GENERAL LAWRENCE J LOGAN WAY,NaN,NaN,"(0.00000000, 0.00000000)",773.15,16450,0.047,0,Other,000000000000,0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555
3,202080065,3301,VERBAL DISPUTE,C6,186,0,2020-10-29 16:30:00,2020,10,Thursday,16,NaN,BOSTON ST,42.327827,-71.058000,"(42.32782700, -71.05800000)",773.15,16450,0.047,0,Other,drt2z191kwfg,0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555
4,202079731,3301,VERBAL DISPUTE,C6,936,0,2020-10-28 13:52:00,2020,10,Wednesday,13,NaN,ORTON-MAROTTA WAY,42.337318,-71.053832,"(42.33731800, -71.05383200)",773.15,16450,0.047,0,Other,drt2z5k0v8ng,0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555


In [ ]:
# boston_crime_2020_df_new.to_csv(data_path + 'boston_crime_2020.csv', index=False)

In [21]:
boston_crime_socio_eco.to_csv(data_path + 'data_prelim/boston_crime_socio_eco.csv', index=False)